In [4]:
# 📦 Install libraries (uncomment if needed)
# !pip install transformers scikit-learn pandas torch

import pandas as pd
import torch
from sklearn.model_selection import train_test_split
from torch.utils.data import Dataset, DataLoader
from transformers import BertTokenizer, BertForSequenceClassification
from torch.optim import AdamW
from transformers import get_scheduler
from tqdm import tqdm

# ✅ Step 1: Load your dataset
df = pd.read_csv("sentiment_analysis.csv")
print("🟢 Loaded dataset with columns:", df.columns)

# ✅ Step 2: Encode sentiment labels (e.g., positive → 0, negative → 1)
label_map = {label: i for i, label in enumerate(df['sentiment'].unique())}
df['label'] = df['sentiment'].map(label_map)

# ✅ Step 3: Split into train and validation
train_texts, val_texts, train_labels, val_labels = train_test_split(
    df['text'].tolist(), df['label'].tolist(), test_size=0.2, random_state=42
)

# ✅ Step 4: Tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

# ✅ Step 5: Dataset class
class SentimentDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, max_length=128):
        self.encodings = tokenizer(texts, truncation=True, padding=True, max_length=max_length)
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

# ✅ Step 6: Create datasets and loaders
train_dataset = SentimentDataset(train_texts, train_labels, tokenizer)
val_dataset = SentimentDataset(val_texts, val_labels, tokenizer)

train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=16)

# ✅ Step 7: Model initialization
model = BertForSequenceClassification.from_pretrained(
    'bert-base-uncased',
    num_labels=len(label_map)
)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# ✅ Step 8: Optimizer and Scheduler
optimizer = AdamW(model.parameters(), lr=5e-5)
num_epochs = 50
num_training_steps = num_epochs * len(train_loader)
lr_scheduler = get_scheduler("linear", optimizer=optimizer, num_warmup_steps=0, num_training_steps=num_training_steps)

# ✅ Step 9: Training loop
for epoch in range(num_epochs):
    model.train()
    total_loss = 0
    progress_bar = tqdm(train_loader, desc=f"Epoch {epoch+1}")

    for batch in progress_bar:
        batch = {k: v.to(device) for k, v in batch.items()}
        outputs = model(**batch)
        loss = outputs.loss
        total_loss += loss.item()

        loss.backward()
        optimizer.step()
        lr_scheduler.step()
        optimizer.zero_grad()

        progress_bar.set_postfix(loss=loss.item())

    print(f"✅ Epoch {epoch+1} complete. Avg Loss: {total_loss / len(train_loader):.4f}")

# ✅ Step 10: Save model and tokenizer
model.save_pretrained("./sentiment_model")
tokenizer.save_pretrained("./sentiment_model")
print("✅ Model training complete and saved to ./sentiment_model")


🟢 Loaded dataset with columns: Index(['Year', 'Month', 'Day', 'Time of Tweet', 'text', 'sentiment',
       'Platform'],
      dtype='object')


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Epoch 1: 100%|██████████| 25/25 [00:03<00:00,  6.52it/s, loss=0.829]


✅ Epoch 1 complete. Avg Loss: 0.9633


Epoch 2: 100%|██████████| 25/25 [00:03<00:00,  6.61it/s, loss=0.601]


✅ Epoch 2 complete. Avg Loss: 0.5591


...

✅ Epoch 48 complete. Avg Loss: 0.0054


Epoch 49: 100%|██████████| 25/25 [00:03<00:00,  6.28it/s, loss=0.000571]


✅ Epoch 49 complete. Avg Loss: 0.0050


Epoch 50: 100%|██████████| 25/25 [00:03<00:00,  6.30it/s, loss=0.0518]


✅ Epoch 50 complete. Avg Loss: 0.0047
✅ Model training complete and saved to ./sentiment_model


In [8]:
# ✅ Step 10: Save model and tokenizer
model.save_pretrained("./sentiment_model")
tokenizer.save_pretrained("./sentiment_model")
print("✅ Model training complete and saved to ./sentiment_model")

✅ Model training complete and saved to ./sentiment_model


In [9]:
# ✅ Step 11: Load the trained model and tokenizer
loaded_model = BertForSequenceClassification.from_pretrained("./sentiment_model")
loaded_tokenizer = BertTokenizer.from_pretrained("./sentiment_model")
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
loaded_model.to(device)


print("✅ Model and tokenizer loaded successfully.")

# ✅ Step 14: Test with random text
def predict_sentiment(text):
    inputs = loaded_tokenizer(text, return_tensors="pt", truncation=True, padding=True, max_length=128)
    inputs = {k: v.to(device) for k, v in inputs.items()}

    with torch.no_grad():
        outputs = loaded_model(**inputs)
        logits = outputs.logits
        predictions = torch.argmax(logits, dim=-1)

    # Get the sentiment label back from the mapped index
    # Reverse the label_map to get label name from index
    reverse_label_map = {i: label for label, i in label_map.items()}
    predicted_label = reverse_label_map[predictions.item()]
    return predicted_label

# Example usage:
test_text = "This is a great product! I love it."
sentiment = predict_sentiment(test_text)
print(f"The sentiment of the text '{test_text}' is: {sentiment}")

test_text_2 = "I am not happy with the service."
sentiment_2 = predict_sentiment(test_text_2)
print(f"The sentiment of the text '{test_text_2}' is: {sentiment_2}")

test_text_3 = "It is what it is."
sentiment_3 = predict_sentiment(test_text_3)
print(f"The sentiment of the text '{test_text_3}' is: {sentiment_3}")

✅ Model and tokenizer loaded successfully.
The sentiment of the text 'This is a great product! I love it.' is: positive
The sentiment of the text 'I am not happy with the service.' is: negative
The sentiment of the text 'It is what it is.' is: neutral


In [14]:
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

# ✅ Step 12: Evaluation loop
loaded_model.eval()
val_labels_list = []
val_preds_list = []

with torch.no_grad():
    for batch in val_loader:
        batch = {k: v.to(device) for k, v in batch.items()}
        outputs = loaded_model(**batch)
        logits = outputs.logits
        predictions = torch.argmax(logits, dim=-1)

        val_labels_list.extend(batch['labels'].cpu().numpy())
        val_preds_list.extend(predictions.cpu().numpy())

# ✅ Step 13: Calculate and print metrics
accuracy = accuracy_score(val_labels_list, val_preds_list)
precision, recall, f1, _ = precision_recall_fscore_support(val_labels_list, val_preds_list, average='weighted')

print(f"✅ Validation Metrics:")
print(f"Accuracy: {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1 Score: {f1:.4f}")

✅ Validation Metrics:
Accuracy: 0.8600
Precision: 0.8645
Recall: 0.8600
F1 Score: 0.8614


In [12]:
from google.colab import drive
import shutil
import os

# Mount Google Drive
drive.mount('/content/drive')

# Define the source and destination paths
source_dir = "/content/sentiment_model"
destination_dir = "/content/drive/MyDrive/model" # You can change the destination folder name

# Create the destination directory if it doesn't exist
if not os.path.exists(destination_dir):
    os.makedirs(destination_dir)

# Copy the model directory to Google Drive
try:
    shutil.copytree(source_dir, destination_dir)
    print(f"✅ Model saved successfully to {destination_dir}")
except FileExistsError:
    print(f"Directory '{destination_dir}' already exists. Model may have been saved previously.")
except Exception as e:
    print(f"❌ Error saving model: {e}")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Directory '/content/drive/MyDrive/model' already exists. Model may have been saved previously.
